In [1]:
from sfm.data.sci_data.SFMDecTokenizer import SFMDecTokenizer
import torch
import os

/anaconda/envs/SFM_shufxi_7bv2_ft/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-04-12 06:11:11,641] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


load tokenizer

In [2]:
tokenizer_home = '/home/yinxia/hai1/ds_dataset/llama2/llama-2-7b'
tokenizer = SFMDecTokenizer.from_pretrained(
    tokenizer_home,
    prot_spm_path='/home/yinxia/blob1.v2/shufxi/data/scigpt/ur50bpe/bpe',
    dna_spm_path='/home/yinxia/blob1.v2/shufxi/data/scigpt/dnabpe/bpe',
    rna_spm_path='/home/yinxia/blob1.v2/shufxi/data/scigpt/rnabpe/bpe',
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'SFMDecTokenizer'.
You are using the default legacy behaviour of the <class 'sfm.data.sci_data.SFMDecTokenizer.SFMDecTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


[2024-04-12 06:11:15.502][INFO]: Loading protein sentencepiece model from /home/yinxia/blob1.v2/shufxi/data/scigpt/ur50bpe/bpe.model and /home/yinxia/blob1.v2/shufxi/data/scigpt/ur50bpe/bpe.vocab
[2024-04-12 06:11:15.561][INFO]: Loading DNA sentencepiece model from /home/yinxia/blob1.v2/shufxi/data/scigpt/dnabpe/bpe.model and /home/yinxia/blob1.v2/shufxi/data/scigpt/dnabpe/bpe.vocab
[2024-04-12 06:11:15.632][INFO]: Loading RNA sentencepiece model from /home/yinxia/blob1.v2/shufxi/data/scigpt/rnabpe/bpe.model and /home/yinxia/blob1.v2/shufxi/data/scigpt/rnabpe/bpe.vocab
[2024-04-12 06:11:16.102][INFO]: Tokenizer has 40014 tokens


In [3]:
tok = tokenizer.tokenize('<mol>c1ccccc1N</mol>')
print(tok)
tok = tokenizer.tokenize('<protein>EMMFEQTFKNID</protein>')
print(tok)
tok = tokenizer.tokenize('<dna>AUCGAAA</dna>')
print(tok)
tok = tokenizer.tokenize('The <protein>EMMFEQTFKNID</protein> and the compound <mol>CCC</mol> can interact with each other.')
print(tok)

['<mol>', '<m>c', '<m>1', '<m>c', '<m>c', '<m>c', '<m>c', '<m>c', '<m>1', '<m>N', '</mol>']
['<protein>', '<a>E', '<a>M', '<a>M', '<a>FEQ', '<a>TFK', '<a>NID', '</protein>']
['<dna>', '<d>A', '<d>U', '<d>C', '<d>GAAA', '</dna>']
['▁The', '<protein>', '<a>E', '<a>M', '<a>M', '<a>FEQ', '<a>TFK', '<a>NID', '</protein>', '▁and', '▁the', '▁comp', 'ound', '<mol>', '<m>C', '<m>C', '<m>C', '</mol>', '▁can', '▁interact', '▁with', '▁each', '▁other', '.']


In [5]:
# Currently, we have three models

# 1. general pre-trained model, no instruction tuning
ckpt_home = r'/home/yinxia/hai1/shufxi/scigpt/7bv3/stageB/global_step32999'

# 2. dedicate protein model, which is continued pre-trained from the `1. general model`
# ckpt_home = r'/home/yinxia/blob1.v2/shufxi/scigpt/7bv3/prot/20240227113241/global_step3905'

# 3. simply run instruction tuning on our collected dataset
# ckpt_home = r'/home/yinxia/blob1.v2/shufxi/scigpt/7bv3/inst/20240301131100/global_step4753'

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def show_ckpt(name, ckpt):
    for k, v in ckpt.items():
        if 'dummy' not in k:
            print(name, k, v.shape)

model = AutoModelForCausalLM.from_pretrained(tokenizer_home)

model_dict = model.state_dict()
ckpt_dict = {}
layer0 = torch.load(os.path.join(ckpt_home, "layer_00-model_states.pt"), map_location=torch.device("cpu"))
ckpt_dict['model.embed_tokens.weight'] = layer0['embed_tokens.weight']
show_ckpt('layer0', layer0)

for l in range(0, 32):
    l_index = str(l + 1).zfill(2)
    layer = torch.load(os.path.join(ckpt_home, f"layer_{l_index}-model_states.pt"), map_location=torch.device("cpu"))
    show_ckpt(l_index, layer)
    for k in layer:
        if "dummy" in k or 'rotary_emb' in k:
            continue
        ckpt_dict[f"model.layers.{l}.{k}"] = layer[k]
layer = torch.load(os.path.join(ckpt_home, "layer_33-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["model.norm.weight"] = layer["norm.weight"]

layer = torch.load(os.path.join(ckpt_home, "layer_34-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["lm_head.weight"] = layer["lm_head.weight"]
model_dict.update(ckpt_dict)

model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(model_dict)

Loading checkpoint shards: 100%|██████████| 2/2 [02:28<00:00, 74.18s/it] 


layer0 embed_tokens.weight torch.Size([40014, 4096])
01 self_attn.q_proj.weight torch.Size([4096, 4096])
01 self_attn.k_proj.weight torch.Size([4096, 4096])
01 self_attn.v_proj.weight torch.Size([4096, 4096])
01 self_attn.o_proj.weight torch.Size([4096, 4096])
01 mlp.gate_proj.weight torch.Size([11008, 4096])
01 mlp.up_proj.weight torch.Size([11008, 4096])
01 mlp.down_proj.weight torch.Size([4096, 11008])
01 input_layernorm.weight torch.Size([4096])
01 post_attention_layernorm.weight torch.Size([4096])
02 self_attn.q_proj.weight torch.Size([4096, 4096])
02 self_attn.k_proj.weight torch.Size([4096, 4096])
02 self_attn.v_proj.weight torch.Size([4096, 4096])
02 self_attn.o_proj.weight torch.Size([4096, 4096])
02 mlp.gate_proj.weight torch.Size([11008, 4096])
02 mlp.up_proj.weight torch.Size([11008, 4096])
02 mlp.down_proj.weight torch.Size([4096, 11008])
02 input_layernorm.weight torch.Size([4096])
02 post_attention_layernorm.weight torch.Size([4096])
03 self_attn.q_proj.weight torch.Size

<All keys matched successfully>

In [7]:
model = model.cuda()

About the usage of fragment-based generation

In [38]:
from rdkit import Chem, RDLogger
RDLogger.DisableLog('rdApp.error')

def reconstruct_molecule_from_fragments(fragments):  
    # Initialize an empty molecule to store the reconstructed molecule  
    reconstructed_mol = Chem.RWMol()  
    # Combine the fragments into a single molecule  
    combined_mol = fragments[0]  
    for fragment in fragments[1:]:  
        combined_mol = Chem.CombineMols(combined_mol, fragment)  
    
    # Convert the combined molecule to an RWMol  
    reconstructed_mol = Chem.RWMol(combined_mol)  
    # Find the dummy atoms and their corresponding neighbors  
    
    dummy_atoms = []  
    for atom_idx, atom in enumerate(reconstructed_mol.GetAtoms()):  
        if atom.GetAtomicNum() == 0:  
            neighbors = [x.GetIdx() for x in atom.GetNeighbors()]  
            dummy_atoms.append((atom_idx, atom.GetAtomMapNum(), neighbors[0] ))  
  
    # Sort dummy atoms by atom map number  
    dummy_atoms.sort(key=lambda x: x[1])  
  
    # Connect the fragments using the dummy atoms and remove them  
    for i in range(0, len(dummy_atoms), 2):  
        atom_idx1, map_num1, neighbor1 = dummy_atoms[i]  
        atom_idx2, map_num2, neighbor2 = dummy_atoms[i + 1]  
  
        if map_num1 == map_num2:  
            # Add a bond between the atoms connected to the dummy atoms  
            reconstructed_mol.AddBond(neighbor1, neighbor2, Chem.rdchem.BondType.SINGLE)  
  
    dummy_atoms = [e[0] for e in dummy_atoms]
    dummy_atoms = sorted(dummy_atoms, reverse=True)
    for atomidx in dummy_atoms: 
        reconstructed_mol.RemoveAtom(atomidx)  

    for atom in reconstructed_mol.GetAtoms():  
        atom.SetAtomMapNum(0)  
  
    return reconstructed_mol


def extract_fragment_output(output):
    try:
        partA = re.search('<fragA>(.*?)</fragA>', output)
        partB = re.search('<fragB>(.*?)</fragB>', output)
        smi_A = partA.group(1).replace('<m>', '').replace(' ', '').strip()
        smi_B = partB.group(1).replace('<m>', '').replace(' ', '').strip()

        segs = []
        for s in smi_A.split('.') + smi_B.split('.'):
            s = s.strip()
            if s == '':
                continue
            segs.append(s)

        mol_frag = [Chem.MolFromSmiles(s) for s in segs]
        m = reconstruct_molecule_from_fragments(mol_frag)
        reconstruct_smi = Chem.MolToSmiles(m)
        return reconstruct_smi
    except:
        return None

In [48]:
from rdkit import Chem, RDLogger
import re 

input_ids = tokenizer('<fragA>O/N=C(\C[*:100000])c1cc(Cl)sc1Cl.c1cncc([*:100001])c1</fragA>', return_tensors="pt").input_ids.cuda()

use_sample = True
if use_sample:
    output = model.generate(
        input_ids,
        max_new_tokens=300,
        num_return_sequences=4,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
else:
    output = model.generate(
        input_ids,
        num_beams=4,
        max_new_tokens=300,
        num_return_sequences=4,
        do_sample=False,
    )

decoded_seq = set()
for o in output:
    s = tokenizer.decode(o)
    r = extract_fragment_output(s)
    if r is not None:
        decoded_seq.add(r)
        print(r)

O/N=C(\Cc1ccc(-c2cccnc2)cc1)c1cc(Cl)sc1Cl
O/N=C(\Cc1nc(-c2cccnc2)cs1)c1cc(Cl)sc1Cl
O/N=C(\Cc1ccc(-c2cccnc2)cc1)c1cc(Cl)sc1Cl
O/N=C(\Cc1ccc(-c2cccnc2)cc1)c1cc(Cl)sc1Cl


Forward prediction

In [50]:
reactants = r"CC(=O)N1CC2CC2(c2ccc([N+]([O-])=O)cc2)C1"

input_str = f'<reactants>{reactants}</reactants><product>'
input_ids = tokenizer(input_str, return_tensors="pt").input_ids.cuda()

output = model.generate(
    input_ids,
    num_beams=4,
    max_new_tokens=300,
    num_return_sequences=4,
    do_sample=False,
)

for o in output:
    s = tokenizer.decode(o)
    s = re.search('<product>(.*?)</product>', s).group(1)
    s = s.replace('<m>', '').replace(' ', '').strip()
    print(s)


 <m>C <m>C <m>( <m>= <m>O <m>) <m>N <m>1 <m>C <m>C <m>2 <m>C <m>C <m>2 <m>( <m>c <m>2 <m>c <m>c <m>c <m>( <m>N <m>) <m>c <m>c <m>2 <m>) <m>C <m>1 
 <m>C <m>C <m>( <m>= <m>O <m>) <m>N <m>1 <m>C <m>[C@@H] <m>2 <m>C <m>[C@] <m>2 <m>( <m>c <m>2 <m>c <m>c <m>c <m>( <m>N <m>) <m>c <m>c <m>2 <m>) <m>C <m>1 
 <m>C <m>C <m>( <m>= <m>O <m>) <m>N <m>1 <m>C <m>[C@H] <m>2 <m>C <m>[C@] <m>2 <m>( <m>c <m>2 <m>c <m>c <m>c <m>( <m>N <m>) <m>c <m>c <m>2 <m>) <m>C <m>1 
 <m>C <m>C <m>( <m>= <m>O <m>) <m>N <m>1 <m>C <m>[C@H] <m>2 <m>C <m>[C@@] <m>2 <m>( <m>c <m>2 <m>c <m>c <m>c <m>( <m>N <m>) <m>c <m>c <m>2 <m>) <m>C <m>1 
